In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

Data Preparation

Load dataset

We can apply the same flow for:

MNIST,
FashionMNIST


 **Tensorflow dataset:**

In [2]:
(ds_train, ds_test), ds_info = tfds.load("fashion_mnist", split=["train", "test"],
                                         as_supervised=True, shuffle_files=True,
                                         with_info=True)
ds_info
     

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/60000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/fashion_mnist/3.0.1.incompleteMOS06I/fashion_mnist-train.tfrecord*...:   0%|  …

Generating test examples...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/fashion_mnist/3.0.1.incompleteMOS06I/fashion_mnist-test.tfrecord*...:   0%|   …

Dataset fashion_mnist downloaded and prepared to ~/tensorflow_datasets/fashion_mnist/3.0.1. Subsequent calls will reuse this data.


tfds.core.DatasetInfo(
    name='fashion_mnist',
    full_name='fashion_mnist/3.0.1',
    description="""
    Fashion-MNIST is a dataset of Zalando's article images consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.
    """,
    homepage='https://github.com/zalandoresearch/fashion-mnist',
    data_path='~/tensorflow_datasets/fashion_mnist/3.0.1',
    file_format=tfrecord,
    download_size=29.45 MiB,
    dataset_size=36.42 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{DBLP:journals/corr/abs-1708-07747,
      author  

Keras dataset


In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(*train_images.shape, 1)
test_images = test_images.reshape(*test_images.shape, 1)

ds_train = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
ds_test = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

11490434/11490434 [==============================] - 1s 0us/step


Batchify dataset


In [4]:
input_shape = next(iter(ds_train))[0].shape
num_classes = 10

In [5]:
BATCH_SIZE = 128
ds_train = ds_train.batch(BATCH_SIZE)
ds_test = ds_test.batch(BATCH_SIZE)
ds_train

<BatchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(None,), dtype=tf.uint8, name=None))>

Model Architecture


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

In [7]:
model = Sequential()

model.add(Rescaling(scale=1./255, input_shape=input_shape))

model.add(Conv2D(filters=16, kernel_size=(3, 3), padding="same", activation="relu", input_shape=input_shape))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.1))

model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(128, activation="relu"))
model.add(Dense(num_classes, activation="softmax"))

model.summary()
     

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 28, 28, 1)         0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 16)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 16)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 32)         0         
 2D)                                                    

Training


In [8]:
model.compile(optimizer="rmsprop", loss="sparse_categorical_crossentropy", metrics=["acc"])
model.fit(ds_train, epochs=20, validation_data=ds_test)
 

Epoch 1/20
469/469 [==============================] - 45s 94ms/step - loss: 0.2789 - acc: 0.9119 - val_loss: 0.0854 - val_acc: 0.9700
Epoch 2/20
469/469 [==============================] - 45s 96ms/step - loss: 0.0787 - acc: 0.9750 - val_loss: 0.0418 - val_acc: 0.9864
Epoch 3/20
469/469 [==============================] - 43s 92ms/step - loss: 0.0561 - acc: 0.9823 - val_loss: 0.0310 - val_acc: 0.9886
Epoch 4/20
469/469 [==============================] - 43s 93ms/step - loss: 0.0456 - acc: 0.9857 - val_loss: 0.0316 - val_acc: 0.9894
Epoch 5/20
469/469 [==============================] - 44s 94ms/step - loss: 0.0391 - acc: 0.9882 - val_loss: 0.0286 - val_acc: 0.9906
Epoch 6/20
469/469 [==============================] - 44s 93ms/step - loss: 0.0348 - acc: 0.9892 - val_loss: 0.0247 - val_acc: 0.9921
Epoch 7/20
469/469 [==============================] - 44s 93ms/step - loss: 0.0316 - acc: 0.9899 - val_loss: 0.0225 - val_acc: 0.9927
Epoch 8/20
469/469 [==============================] - 45s 96ms